# Bulk Data Processing
This notebook is for processing and saving data across multiple subjects in bulk.

# Setup
Run this first to import everything and set up notebook

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import logging
import os
import pickle

import numpy as np
from foraging import utils

import utils.beliefs
import utils.data

# Filter out annoying matplotlib logs
mlogger = logging.getLogger('matplotlib')
mlogger.setLevel(logging.WARNING)

EXPERIMENTS_DIR = '../data/experiments'
SAVE_DIR = '../data/analysis'
SEED = 42

# Load experiment data
Load matlab datafiles into a pandas Dataframe

In [2]:
df = utils.data.make_dataframe(EXPERIMENTS_DIR)
df = utils.data.exclusion_criteria(df)
df.head()

session id  \
subject session block push # stimulus type shape kappa week day               
dylan   1       2     1      probability   1     0.1   M           20211206   
                      2      probability   1     0.1   M           20211206   
                      3      probability   1     0.1   M           20211206   
                      4      probability   1     0.1   M           20211206   
                      5      probability   1     0.1   M           20211206   

                                                                 _session  \
subject session block push # stimulus type shape kappa week day             
dylan   1       2     1      probability   1     0.1   M               20   
                      2      probability   1     0.1   M               20   
                      3      probability   1     0.1   M               20   
                      4      probability   1     0.1   M               20   
                      5      probability   1     0.1   M               20   

                                                                 schedule  \
subject session block push # stimulus type shape kappa week day             
dylan   1       2     1      probability   1     0.1   M             15.0   
                      2      probability   1     0.1   M             15.0   
                      3      probability   1     0.1   M             15.0   
                      4      probability   1     0.1   M             21.0   
                      5      probability   1     0.1   M             21.0   

                                                                 box  \
subject session block push # stimulus type shape kappa week day        
dylan   1       2     1      probability   1     0.1   M           1   
                      2      probability   1     0.1   M           1   
                      3      probability   1     0.1   M           1   
                      4      probability   1     0.1   M           2   
                      5      probability   1     0.1   M           2   

                                                                 push times  \
subject session block push # stimulus type shape kappa week day               
dylan   1       2     1      probability   1     0.1   M              8.465   
                      2      probability   1     0.1   M              9.468   
                      3      probability   1     0.1   M             11.667   
                      4      probability   1     0.1   M             17.125   
                      5      probability   1     0.1   M             19.904   

                                                                 same-box push intervals  \
subject session block push # stimulus type shape kappa week day                            
dylan   1       2     1      probability   1     0.1   M                           8.465   
                      2      probability   1     0.1   M                           1.003   
                      3      probability   1     0.1   M                           2.199   
                      4      probability   1     0.1   M                          17.125   
                      5      probability   1     0.1   M                           2.779   

                                                                 reward outcomes  \
subject session block push # stimulus type shape kappa week day                    
dylan   1       2     1      probability   1     0.1   M                   False   
                      2      probability   1     0.1   M                   False   
                      3      probability   1     0.1   M                   False   
                      4      probability   1     0.1   M                   False   
                      5      probability   1     0.1   M                   False   

                                                                 reward intervals  \
subject session block push # stimulus type shape kapp

# Compute beliefs without color
Compute the posterior for all blocks where the color cue was uninformative

In [3]:
supp = np.arange(1, 30)
df_nocol = utils.data.filter_df(df, {'kappa': 0})
posteriors, err_beliefs = utils.data.process_blocks(df_nocol, utils.beliefs.compute_posteriors, supp, use_tqdm=True)

100%|██████████| 26/26 [00:00<00:00, 113.46it/s]


In [4]:
func = lambda df, index: utils.beliefs.compute_latent_beliefs_over_time(df, index, posteriors[index], dt=0.5,
                                                                        padding_time=0.5)
latent_beliefs_time, err_latent_time = utils.data.process_blocks(df_nocol, func, use_tqdm=True)

100%|██████████| 26/26 [00:00<00:00, 531.53it/s]


In [5]:
func = lambda df, index: utils.beliefs.compute_joint_beliefs(df, index, posteriors[index])
joint_beliefs, err_joint = utils.data.process_blocks(df_nocol, func, use_tqdm=True)

100%|██████████| 26/26 [00:01<00:00, 24.95it/s]


In [6]:
func = lambda df, index: utils.beliefs.compute_reward_beliefs(df, index, posteriors[index])
reward_beliefs, err_reward = utils.data.process_blocks(df_nocol, func, use_tqdm=True)

100%|██████████| 26/26 [00:01<00:00, 22.19it/s]



## Perfect model
Compute the exact reward probabilities under a perfect model of the boxes (one where the schedules are known exactly)

In [7]:
reward_probabilities, err_reward_probabilities = utils.data.process_blocks(df_nocol,
                                                                           utils.beliefs.compute_reward_probabilities,
                                                                           use_tqdm=True)

100%|██████████| 26/26 [00:00<00:00, 50.98it/s]


# Save data

In [8]:
with open(os.path.join(SAVE_DIR, 'bulk_beliefs.pkl'), 'wb') as f:
    pickle.dump({
        'data': {
            'posteriors': posteriors,
            'latent_beliefs_over_time': latent_beliefs_time,
            'joint_beliefs': joint_beliefs,
            'reward_beliefs': reward_beliefs,
            'reward probabilities': reward_probabilities
        },
        'error': {
            'err_schedule_beliefs': err_beliefs,
            'err_joint_beliefs': err_joint,
            'err_reward_beliefs': err_reward,
            'err_reward probabilities': err_reward_probabilities,
        },
        'schedule candidates': supp,
        'dt': 0.5,
        'padding_time': 0.5
    }, f)